In [8]:
import json
import pathlib
import pandas as pd
import re

# Plaka -> İl eşlemesi
PLAKA_TO_IL = {
    1:"Adana",2:"Adıyaman",3:"Afyonkarahisar",4:"Ağrı",5:"Amasya",6:"Ankara",7:"Antalya",8:"Artvin",9:"Aydın",
    10:"Balıkesir",11:"Bilecik",12:"Bingöl",13:"Bitlis",14:"Bolu",15:"Burdur",16:"Bursa",17:"Çanakkale",18:"Çankırı",
    19:"Çorum",20:"Denizli",21:"Diyarbakır",22:"Edirne",23:"Elazığ",24:"Erzincan",25:"Erzurum",26:"Eskişehir",
    27:"Gaziantep",28:"Giresun",29:"Gümüşhane",30:"Hakkâri",31:"Hatay",32:"Isparta",33:"Mersin",34:"İstanbul",
    35:"İzmir",36:"Kars",37:"Kastamonu",38:"Kayseri",39:"Kırklareli",40:"Kırşehir",41:"Kocaeli",42:"Konya",
    43:"Kütahya",44:"Malatya",45:"Manisa",46:"Kahramanmaraş",47:"Mardin",48:"Muğla",49:"Muş",50:"Nevşehir",
    51:"Niğde",52:"Ordu",53:"Rize",54:"Sakarya",55:"Samsun",56:"Siirt",57:"Sinop",58:"Sivas",59:"Tekirdağ",
    60:"Tokat",61:"Trabzon",62:"Tunceli",63:"Şanlıurfa",64:"Uşak",65:"Van",66:"Yozgat",67:"Zonguldak",68:"Aksaray",
    69:"Bayburt",70:"Karaman",71:"Kırıkkale",72:"Batman",73:"Şırnak",74:"Bartın",75:"Ardahan",76:"Iğdır",
    77:"Yalova",78:"Karabük",79:"Kilis",80:"Osmaniye",81:"Düzce"
}

def slugify_tr(s: str) -> str:
    """Türkçe karakterleri sadeleştirip, boşluk ve noktalama yerine tire koyar."""
    tr_map = str.maketrans({
        "ç":"c","Ç":"c","ğ":"g","Ğ":"g","ı":"i","İ":"i",
        "ö":"o","Ö":"o","ş":"s","Ş":"s","ü":"u","Ü":"u"
    })
    s = (s or "").translate(tr_map).lower()
    s = re.sub(r"[^a-z0-9]+", "-", s).strip("-")
    return s or "cikti"

# JSON klasörünün yolu
json_dir = pathlib.Path("C:/jsons")
out_dir = pathlib.Path("C:/out")
out_dir.mkdir(parents=True, exist_ok=True)

rows = []
indicator_name = None  # gosterge_ad burada tutulacak

# Tüm JSON dosyalarını oku
for file in json_dir.glob("*.json"):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)

    # İlk dosyadan gosterge_ad al
    if indicator_name is None:
        indicator_name = data.get("gosterge_ad")

    years = [int(y) for y in data.get("tarihler", [])]

    for item in data.get("veriler", []):
        code = int(item["duzeyKodu"])
        il = PLAKA_TO_IL.get(code, f"Bilinmeyen-{code}")
        values = [int(v) for v in item.get("veri", [])]

        for year, value in zip(years, values):
            rows.append({"il_kodu": code, "il": il, "yil": year, "nufus": value})

# DataFrame oluştur
df = pd.DataFrame(rows)

# Geniş forma pivotla
wide = df.drop_duplicates(subset=["il_kodu","il","yil"], keep="first") \
         .pivot_table(index=["il_kodu","il"], columns="yil", values="nufus") \
         .reset_index()

wide.columns.name = None  # Pivot sonrası sütun ismini temizle

# Dosya adı için slug hazırla (örn. "Girişim sayıları" -> "girisim-sayilari")
base = slugify_tr(indicator_name)

# CSV ve Excel kaydet
# wide.to_csv(out_dir / f"{base}.csv", index=False, encoding="utf-8")
with pd.ExcelWriter(out_dir / f"{base}.xlsx", engine="xlsxwriter") as writer:
    wide.to_excel(writer, sheet_name=indicator_name, index=False)

print(f"Dosya kaydedildi: {out_dir} / {base}.xlsx")

Dosya kaydedildi: C:\out / girisim-sayilari.xlsx
